### Pre-Procssing Raw Tables

In [0]:
customer_tb = spark.read.table('pei_adb_proj.bronze.customer_raw')
product_tb = spark.read.table('pei_adb_proj.bronze.product_raw')
order_tb = spark.read.table('pei_adb_proj.bronze.order_raw')

In [0]:
# order_tb.display()
customer_tb.display()
# product_tb.display()

#####Clean Data 

In [0]:
from pyspark.sql.functions import regexp_replace, col, to_date, trim, when

customer_cleaned = customer_tb.withColumn("Customer_Name",regexp_replace(col("Customer_Name"), r"[^a-zA-Z\s]", ""))
customer_cleaned = customer_cleaned.withColumn("Customer_Name",trim(col("Customer_Name")))\
                                .withColumn('Customer_Name', regexp_replace('Customer_Name', ' +', ' '))\
                                .withColumn("phone",trim(col("phone")))\
                                .withColumn('phone', regexp_replace('phone', r'[^0-9]', ''))\
                                .withColumn('phone',when(col('phone').rlike('^[0-9]{10}$'), col('phone')).otherwise(None))\
                                .withColumn('postal_code', regexp_replace('postal_code', '[^0-9]', ''))

invalid_phones = customer_cleaned.filter(~(col("phone").rlike("^[0-9]{10}$")) & col("phone").isNotNull())
assert invalid_phones.count() == 0, "Phone validation failed: Found phones not matching 10 digits."
print("Test Passed: All phone numbers are valid.")

In [0]:
product_cleaned = product_tb.withColumn('Price_Per_Product',regexp_replace('Price_Per_Product', '[^0-9.]', ''))\
                            .withColumn('State', regexp_replace('State',r"[^A-Za-z\s]",""))


#####Standardizating Cleaned Data

In [0]:
customer_stan = customer_cleaned.withColumnRenamed('phone',"Phone")\
                            .withColumnRenamed('email',"Email")\
                            .withColumnRenamed('address',"Address")\
                            .withColumnRenamed('postal_code',"Postal_Code")

customer_stan = customer_stan.withColumn('Phone',col('Phone').cast('long'))\
                             .withColumn('Postal_Code', col('Postal_Code').cast('int'))


In [0]:
product_stan = product_cleaned.withColumn("Price_Per_Product",col("Price_Per_Product").cast('double'))

In [0]:
order_stan = order_tb.withColumn('Order_Date', to_date('Order_Date', 'd/M/yyyy'))\
                   .withColumn('Shipping_Date', to_date('Shipping_Date', 'd/M/yyyy'))\
                    .withColumnRenamed('Row ID',"Row_ID")


#####Validation

In [0]:
valid_email_pattern = r'^[A-Za-z0-9._%+-]+@[A-Za-z]+\.[A-Za-z]{2,}$'

customer_stan = customer_stan.filter(col('Email').rlike(valid_email_pattern))

#####De-Duplication

In [0]:
customer_stan = customer_stan.dropDuplicates(['Customer_ID'])
product_stan = product_stan.dropDuplicates(['Product_ID'])
order_stan = order_stan.dropDuplicates(['Order_ID'])


In [0]:
spark.sql("drop table if exists pei_adb_proj.silver.order_stan_silver")
spark.sql("drop table if exists pei_adb_proj.silver.customer_stan_silver")
spark.sql("drop table if exists pei_adb_proj.silver.product_stan_silver")

In [0]:
customer_stan.write.format("delta").mode("overwrite").saveAsTable("pei_adb_proj.silver.customer_stan_silver")
product_stan.write.format("delta").mode("overwrite").saveAsTable("pei_adb_proj.silver.product_stan_silver")
order_stan.write.format("delta").mode("overwrite").saveAsTable("pei_adb_proj.silver.order_stan_silver")



### Test Cases For Pre-processing

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, regexp_replace, trim, when

# Sample DataFrame to test
test_data = [
    Row(Customer_Name="John@123  Smith", phone=" 98765-43210 ", postal_code="700 001"),
    Row(Customer_Name="Jane    Doe!!", phone="1234abc567", postal_code="ABCDE"),
    Row(Customer_Name="   Bob   ", phone="999999999", postal_code="123456")
]

df = spark.createDataFrame(test_data)

# Apply the cleaning logic
customer_cleaned = df.withColumn("Customer_Name", regexp_replace(col("Customer_Name"), r"[^a-zA-Z\s]", "")) \
                     .withColumn("Customer_Name", trim(col("Customer_Name"))) \
                     .withColumn('Customer_Name', regexp_replace('Customer_Name', ' +', ' ')) \
                     .withColumn("phone", trim(col("phone"))) \
                     .withColumn('phone', regexp_replace('phone', r'[^0-9]', '')) \
                     .withColumn('phone', when(col('phone').rlike('^[0-9]{10}$'), col('phone')).otherwise(None)) \
                     .withColumn('postal_code', regexp_replace('postal_code', '[^0-9]', ''))


In [0]:
# Validate special characters are removed
invalid_chars = customer_cleaned.filter(col("Customer_Name").rlike("[^a-zA-Z\s]"))
assert invalid_chars.count() == 0, "Test Case 1 Failed: Special characters not fully removed from Customer_Name."
print("OK Test Case 1 Passed: Special characters removed from Customer_Name.")


In [0]:
# Validate no multiple spaces exist
multiple_spaces = customer_cleaned.filter(col("Customer_Name").rlike(" {2,}"))
assert multiple_spaces.count() == 0, "Test Case 2 Failed: Multiple spaces found in Customer_Name."
print("OK Test Case 2 Passed: Multiple spaces reduced to single space in Customer_Name.")


In [0]:
# Validate no leading or trailing spaces
leading_trailing_spaces = customer_cleaned.filter(
    col("Customer_Name").startswith(" ") | col("Customer_Name").endswith(" ")
)
assert leading_trailing_spaces.count() == 0, "Test Case 3 Failed: Leading or trailing spaces found in Customer_Name."
print("OK Test Case 3 Passed: Leading and trailing spaces removed from Customer_Name.")


In [0]:
# Validate phone numbers contain only digits or are null
invalid_phones = customer_cleaned.filter(
    ~(col("phone").rlike("^[0-9]{10}$")) & col("phone").isNotNull()
)
assert invalid_phones.count() == 0, "Test Case 4 Failed: Invalid phone numbers found."
print("OK Test Case 4 Passed: All phone numbers are valid or null.")


In [0]:
# Validate phones with less/more than 10 digits are set to null
incorrect_phones = customer_cleaned.filter((col("phone").isNotNull()) & (~col("phone").rlike("^[0-9]{10}$")))
assert incorrect_phones.count() == 0, "Test Case 5 Failed: Invalid length phone numbers not set to null."
print("OK Test Case 5 Passed: Incorrect length phone numbers are correctly nullified.")


In [0]:
# Validate postal codes contain only digits
invalid_postal = customer_cleaned.filter(~col("postal_code").rlike("^[0-9]*$"))
assert invalid_postal.count() == 0, "Test Case 6 Failed: Non-numeric characters found in postal codes."
print("OK Test Case 6 Passed: Postal codes cleaned successfully.")


In [0]:
dbutils.notebook.exit("Pre-processing completed successfully")